In [1]:
# import datetime
# import pytz
# import os
# import json
# from ruamel import yaml
# import ruamel
# from collections import defaultdict
# import re
# import pandas as pd
# import numpy as np
# import great_expectations 
# from great_expectations.core.batch import RuntimeBatchRequest
# from great_expectations.checkpoint.checkpoint import SimpleCheckpoint
# from great_expectations.data_context import BaseDataContext
# from great_expectations.data_context.types.base import (
#     DataContextConfig,
#     FilesystemStoreBackendDefaults,
# )
# from great_expectations.profile.user_configurable_profiler  import UserConfigurableProfiler
# import IPython
# import great_expectations as ge
# from typing import TypeVar
# PandasDataFrame = TypeVar('pandas.core.frame.DataFrame')
# class DataQuality():
    
#     def __init__(self, root_directory: str, datasource_name:str , dataframe:PandasDataFrame  , partition_date: datetime.date):
        
#         '''
#         create great expectations context and default runtime datasource
#         '''
        
#         self.root_directory = root_directory
#         self.datasource_name = datasource_name
#         self.expectation_suite_name = f"{datasource_name}_expectation_suite"
#         self.checkpoint_name = f"{datasource_name}_checkpoint"
#         self.dataframe = dataframe
#         self.partition_date = partition_date
        
#         # create data context
#         data_context_config = DataContextConfig( store_backend_defaults=FilesystemStoreBackendDefaults(root_directory=root_directory) )
#         context = BaseDataContext(project_config = data_context_config)
     
#         datasource_yaml = rf"""
#         name: {self.datasource_name}
#         class_name: Datasource
#         module_name: great_expectations.datasource
#         execution_engine:
#           module_name: great_expectations.execution_engine
#           class_name: PandasExecutionEngine
#         data_connectors:
#             default_runtime_data_connector_name:
#                 class_name: RuntimeDataConnector
#                 batch_identifiers:
#                     - default_identifier_name
#         """
#         context.test_yaml_config(datasource_yaml)
#         context.add_datasource(**yaml.load(datasource_yaml, Loader=ruamel.yaml.Loader))
        
#         self.context = context
    
#     def get_context(self):
        
#         '''
#         retrieving data context in case you would like to manually extract / tweak the context by yourself
#         '''
        
#         return self.context
    
#     def get_expectation_suit(self):
        
#         '''
#         retriving the current expectation suite
#         '''
        
#         return self.context.get_expectation_suite(self.expectation_suite_name)
    
#     def create_batch_data(self, df, partition_date: datetime.date):
        
#         '''
#         create runtime batch request from the input dataframe and partition date
#         '''
        
#         batch_request = RuntimeBatchRequest(
#             datasource_name= self.datasource_name,
#             data_connector_name= "default_runtime_data_connector_name",
#             data_asset_name=f"{self.datasource_name}_{self.partition_date.strftime('%Y%m%d')}",
#             batch_identifiers={"default_identifier_name": "default_identifier"},
#             runtime_parameters={"batch_data": df}
#         )
        
#         return batch_request
    
#     def create_expectation_suite_if_not_exist(self):
        
#         '''
#         create expectation suite if not exist
#         '''
        
#         try:
#             # create expectation suite
#             self.context.create_expectation_suite(
#                 expectation_suite_name = self.expectation_suite_name,
#                 overwrite_existing=False
#             )
#         except great_expectations.exceptions.DataContextError as e:
#             print(e)
#         except Error as e:
#             raise e
            
#     def delete_expectation_suite(self):
        
#         '''
#         delete the expectation suite
#         '''
        
#         self.context.delete_expectation_suite(expectation_suite_name = self.expectation_suite_name)
    
#     def get_validator(self, with_profiled=False):
        
#         '''
#         retreiving a validator object for a fine grain adjustment on the expectation suite. and you can add some assert on this
#         such as hight must be more than 0
#         '''
        
#         batch_request = self.create_batch_data(self.dataframe, self.partition_date)
#         self.create_expectation_suite_if_not_exist()

#         validator = self.context.get_validator(
#             batch_request = batch_request,
#             expectation_suite_name = self.expectation_suite_name,
#         )
        
#         if with_profiled:

#             # build expectation with profiler
#             not_null_only = True
#             table_expectations_only = False

#             profiler = UserConfigurableProfiler(
#                 profile_dataset = validator,
#                 not_null_only = not_null_only,
#                 table_expectations_only = table_expectations_only
#             )

#             suite = profiler.build_suite()

#             # save validation
#             validator.save_expectation_suite(discard_failed_expectations=False)
        
#         return validator
    
#     def create_checkpoint_if_not_exist(self):
        
#         '''
#         create checkpoint if not exist.
#         '''
        
#         try:
#             self.context.get_checkpoint(self.checkpoint_name)
#             print(f'{self.checkpoint_name} is already created')
                    
#         except great_expectations.exceptions.CheckpointNotFoundError:
            
#             checkpoint_config = {
#                 "name": self.checkpoint_name,
#                 "config_version": 1.0,
#                 "class_name": "SimpleCheckpoint",
#                 "run_name_template": "%Y%m%d-%H%M%S",
#             }
#             self.context.test_yaml_config(yaml.dump(checkpoint_config))
#             self.context.add_checkpoint(**checkpoint_config)

#         except Error as e:
#             raise e
            
            
#     def validate_data(self, df=None, partition_date: datetime.date=None):
        
#         '''
#         validate dataset using the input dataset when initiated the class
#         or user provided dataset when calling the method.
#         '''
        
#         if df and partition_date:
#             batch_request = self.create_batch_data(df, partition_date)
#         else:
#             batch_request = self.create_batch_data(self.dataframe, self.partition_date)            
        
#         self.create_checkpoint_if_not_exist()
        
#         # run expectation_suite against data
#         checkpoint_result = self.context.run_checkpoint(
#             checkpoint_name = self.checkpoint_name,
#             validations=[
#                 {
#                     "batch_request": batch_request,
#                     "expectation_suite_name": self.expectation_suite_name,
#                 }
#             ],
#         )
        
#         for k,v in checkpoint_result['run_results'].items():
#             self.render_file = v['actions_results']['update_data_docs']['local_site'].replace('file://', '')
    
#         return checkpoint_result   

In [1]:
import pandas as pd
csv_url = ("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv")
data = pd.read_csv(csv_url, sep=";")

In [2]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
from great_expectation_function.build_expectation_suite import DataQuality

In [3]:
for i in data.columns:
    data[i] =data[i]**2
test_data = data

In [11]:
import datetime
from great_expectation_function.build_expectation_suite import DataQuality
# initial the root directory for great expectation
root_directory = "/great_expectation"
# initial our data quality checker
def data_validation(dataframe, datasource_name:str):
    f = open('/code/notebook/great_expectation_function/myfile.txt', 'r')
    list_datasource = [x for x in f.readlines()]
    if datasource_name not in list_datasource:
        dq_checker = DataQuality(root_directory, datasource_name, dataframe, partition_date = datetime.datetime.now().date())
        # for the first time, create expectation suite with the similar name to your datasource_name
        dq_checker.create_expectation_suite_if_not_exist()
        # build the suite with automatic profiling.
        validator = dq_checker.get_validator(with_profiled=True, append_suit = True)
        # validate your data
        result = dq_checker.validate_data()
    else:
        dq_checker = DataQuality(root_directory, datasource_name, dataframe, partition_date = datetime.datetime.now().date())
        result = dq_checker.validate_data()
    return dq_checker.render_file

In [12]:
data_validation(data, "wine_quality")

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []

wine_quality_checkpoint is already created


Calculating Metrics:   0%|          | 0/183 [00:00<?, ?it/s]

'/great_expectation/uncommitted/data_docs/local_site/validations/wine_quality_expectation_suite/20221122-230757/20221122T230757.078344Z/3d2c7eab00142a1b886d5492d0d8a462.html'

In [13]:
import os
from flask import Flask,render_template
path_to_list = data_validation(data, "wine_quality").split("/")
template_dir = os.path.abspath("/".join(path_to_list[:-1]))
app = Flask('app',template_folder=template_dir)

@app.route('/')
def main():
    return render_template(path_to_list[-1])

app.run(host='0.0.0.0', port=8080)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []

wine_quality_checkpoint is already created


Calculating Metrics:   0%|          | 0/183 [00:00<?, ?it/s]

 * Serving Flask app 'app'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.26.0.4:8080
Press CTRL+C to quit


In [17]:
from great_expectation_function.run import runner

In [18]:
runner()

/usr/local/bin/python3: can't find '__main__' module in '/code/notebook/great_expectation_function'


256

In [20]:
os.system("python3 great_expectation_function/app.py")

Traceback (most recent call last):
  File "/code/notebook/great_expectation_function/app.py", line 5, in <module>
    path_to_list = data_validation(data, "wine_quality").split("/")
NameError: name 'data' is not defined


256

In [1]:
from great_expectation_function.validate import path_to_json

In [3]:
path_to_json(data, "wine_quality")

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []

wine_quality_checkpoint is already created


Calculating Metrics:   0%|          | 0/183 [00:00<?, ?it/s]

In [5]:
from great_expectation_function.run import runner

In [ ]:
runner()

sh: 1: lsof: not found
sh: 1: kill: Usage: kill [-s sigspec | -signum | -sigspec] [pid | job]... or
kill -l [exitstatus]


 * Serving Flask app 'app'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.24.0.4:8080
Press CTRL+C to quit
172.24.0.1 - - [23/Nov/2022 15:51:35] "GET / HTTP/1.1" 200 -
172.24.0.1 - - [23/Nov/2022 15:51:36] "GET /static/images/iterative-dev-loop.png HTTP/1.1" 404 -
172.24.0.1 - - [23/Nov/2022 15:51:36] "GET /static/images/validation_failed_unexpected_values.gif HTTP/1.1" 404 -
172.24.0.1 - - [23/Nov/2022 15:51:36] "GET /static/images/glossary_scroller.gif HTTP/1.1" 404 -


In [3]:
from script.great_expectation_function.validate import path_to_json

In [4]:
path_to_json(data, "wine_quality", True)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []

expectation_suite with name wine_quality_expectation_suite already exists. If you would like to overwrite this expectation_suite, set overwrite_existing=True.


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Creating an expectation suite with the following expectations:

Table-Level Expectations
expect_table_columns_to_match_ordered_list
expect_table_row_count_to_be_between

Expectations by Column
Column Name: alcohol | Column Data Type: FLOAT | Cardinality: MANY
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_median_to_be_between
expect_column_min_to_be_between
expect_column_proportion_of_unique_values_to_be_between
expect_column_quantile_values_to_be_between
expect_column_values_to_be_in_set
expect_column_values_to_be_in_type_list
expect_column_values_to_not_be_null


Column Name: chlorides | Column Data Type: FLOAT | Cardinality: MANY
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_median_to_be_between
expect_column_min_to_be_between
expect_column_proportion_of_unique_values_to_be_between
expect_column_quantile_values_to_be_between
expect_column_values_to_be_in_set
expect_column_values_to_be_in_type_list
expect_column_values_

Calculating Metrics:   0%|          | 0/183 [00:00<?, ?it/s]